In [50]:
import os
from dotenv import load_dotenv

In [51]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if google_api_key:
    print("Gemini API key loaded successfully!")
else:
    print("Failed to load Gemini API key from .env file!")

Gemini API key loaded successfully!


In [52]:
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage
from typing import Annotated

In [53]:
# tool create

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function calculates the target currency value from a given base currency value
    """

    return base_currency_value * conversion_rate

In [54]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745625602,
 'time_last_update_utc': 'Sat, 26 Apr 2025 00:00:02 +0000',
 'time_next_update_unix': 1745712002,
 'time_next_update_utc': 'Sun, 27 Apr 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4368}

In [55]:
convert.invoke({'base_currency_value': 10, 'conversion_rate': 85.16})

851.5999999999999

### Tool Binding

In [56]:
llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-1.5-pro")

In [57]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

### Tool Calling

In [58]:
messages = [HumanMessage("what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR")]

In [59]:
ai_message = llm_with_tools.invoke(messages)

In [60]:
messages.append(ai_message)

In [61]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '5cdc8826-2985-48de-8ec1-4f0f1fb9f894',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10.0},
  'id': 'f76175dc-0e46-4de1-aefa-73c8f7c78b69',
  'type': 'tool_call'}]

In [62]:
import json

for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # print(tool_message1)
        # fetch the conversion rate
        conversion_rate = (json.loads(tool_message1.content)['conversion_rate'])
        # append this tool message to message list
        messages.append(tool_message1)
    # execute the 2nd tool using the conversion rate from tool1
    if tool_call['name'] == 'convert':
        # fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [63]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-3ea91f3d-acc5-45f3-acf6-cc8d97a3936c-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '5cdc8826-2985-48de-8ec1-4f0f1fb9f894', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10.0, 'conversion_rate': 85.4368}, 'id': 'f76175dc-0e46-4de1-aefa-73c8f7c78b69', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 19, 'total_tokens': 105, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "succ

In [64]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 85.4368. So, 10 USD is equivalent to 854.368 INR.'